In [ ]:
import gmsh

In [ ]:
import sys
%matplotlib inline
import matplotlib.pyplot as plt
from sympy import solve, symbols, nonlinsolve
from math import pi, sin, cos, asin, atan
import numpy as np

In [ ]:
gmsh.initialize()
gmsh.model.add("horn-lines")

In [ ]:
r = 1
a = 1/15
b = 3/10
t = a/2

freq=1000
c_air=340
l=c_air/freq
lc=l/12

In [ ]:
omega = atan(a/r)
omega2 = atan((a+t)/r)
theta = atan((b-a)/(r/2))
points = [
        (-r,0),                                           # 1 left corner
        (-r+a*sin(omega), a*cos(omega)),                  # 2 horn left low
        (-r/2, a),                                        # 3 horn mid low
        (0, b),                                           # 4 horn right low
        (0   +t*cos(pi/2+theta), b+t*sin(pi/2+theta)),    # 5 horn right high
        (-r/2+t*cos(pi/2+theta), a+t*sin(pi/2+theta)),    # 6 horn mid high
        (-r+(a+t)*sin(omega2), 0+(a+t)*cos(omega2)),      # 7 horn left high
        (r,0),                                            # 8 right corner
        (0,0),                                            # 9 circle center
]

In [ ]:
p_points_x = []
p_points_y = []

def add_points(pi):
    p_points_x.append(points[pi-1][0])
    p_points_y.append(points[pi-1][1])
    
add_points(8)
add_points(1)
add_points(2)
add_points(3)
add_points(4)
add_points(5)
add_points(6)
add_points(7)

p_points_x.append(-1)
p_points_y.append(1)
p_points_x.append(1)
p_points_y.append(1)
p_points_x.append(1)
p_points_y.append(0)


plt.plot(p_points_x, p_points_y)

In [ ]:
g_points = []
for p in points:
    g_points.append(gmsh.model.geo.addPoint(p[0], p[1], 0, lc))
    
s_in = gmsh.model.geo.addLine(1, 2)
s_h1 = gmsh.model.geo.addLine(2, 3)
s_h2 = gmsh.model.geo.addLine(3, 4)
s_h3 = gmsh.model.geo.addLine(4, 5)
s_h4 = gmsh.model.geo.addLine(5, 6)
s_h5 = gmsh.model.geo.addLine(6, 7)
s_out = gmsh.model.geo.addCircleArc(7, 9, 8)
s_symm = gmsh.model.geo.addLine(8, 1)

In [ ]:
g_border    = gmsh.model.geo.addCurveLoop([s_in, s_h1, s_h2, s_h3, s_h4, s_h5, s_out, s_symm])
g_inlet     = gmsh.model.geo.addCurveLoop([s_in])
g_outlet    = gmsh.model.geo.addCurveLoop([s_out])
g_symm      = gmsh.model.geo.addCurveLoop([s_symm])
g_hard      = gmsh.model.geo.addCurveLoop([s_h1, s_h2, s_h3, s_h4, s_h5])

g_surface   = gmsh.model.geo.addPlaneSurface([g_border])
print('border {} inlet {} outlet {} symm {} hard {}'.format(g_border, g_inlet, g_outlet, g_symm, g_hard))

gmsh.model.geo.synchronize()

pm_surface = 1
gmsh.model.addPhysicalGroup(dim=2, tags=[g_surface], tag=pm_surface)
gmsh.model.setPhysicalName(dim=2, tag=pm_surface, name="Horn 2D")

pm_inlet = 2
gmsh.model.addPhysicalGroup(dim=1, tags=[s_in], tag=pm_inlet)
gmsh.model.setPhysicalName(dim=1, tag=pm_inlet, name="Boundary Inlet")

pm_outlet = 3
gmsh.model.addPhysicalGroup(dim=1, tags=[s_out], tag=pm_outlet)
gmsh.model.setPhysicalName(dim=1, tag=pm_outlet, name="Boundary Outlet")


gmsh.model.mesh.generate(3)
gmsh.write("horn-line.msh")









In [ ]:
from firedrake import *
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

mesh = Mesh('horn-line.msh', complex_mode=False)


In [ ]:
triplot(mesh)

In [ ]:
g_inlet

In [ ]:
g_outlet